# Introdução


Nesse pequeno tutorial vamos ver um pouco sobre
* Jupyter
* Pandas
* Seaborn
* Scikit-learn
* eli5


# Jupyter

O [Jupyter](https://jupyter.org/) é um ambiente de desenvolvimento para diversas linguagens interpretadas como Python e R.

Permite a execução de "blocos" de código.

Pode se mesclar códigos com explicações.

Tutoriais:
- https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook
- https://www.dataquest.io/blog/jupyter-notebook-tutorial/

Simples olá mundo

In [ ]:
print("Olá mundo!")

Baixando a base de dados que iremos utilizar Iris:

![Iris](http://s5047.pcdn.co/wp-content/uploads/2015/04/iris_petal_sepal.png)

In [ ]:
import urllib.request

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
file = "iris.data"

urllib.request.urlretrieve(url, file)

# Pandas

A biblioteca [Pandas](https://pandas.pydata.org/) permite trabalhar com tabelas em python.

Basicamente um Excel em Python.

Tutoriais:
- https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python
- https://www.dataquest.io/blog/pandas-python-tutorial/

Lendo o arquivo que baixamos antes e colocando nome em suas colunas

In [ ]:
import pandas as pd

feature_names = ["sepal length", "sepal width", "petal length", "petal width"]
class_name = "species"

df = pd.read_csv(file, names=feature_names + [class_name])


O atributo `shape` mostra o formato da tabela em linhas e colunas.
O método `head()` permite selecionar o início da tabela. Útil para conferir uma tabela.

In [ ]:
print(df.shape)
df.head()

É possivel selecionar colunas e linhas.
Aqui selecionamos os elementos unicos na coluna `label`.



In [ ]:
df[class_name].unique()

Aqui já selecionamos a linha com index 2

In [ ]:
df.iloc[2]

`describe()` apresenta algumas informações interessantes de cada coluna com valores númericos.

In [ ]:
df.describe()

A biblioteca [seaborn](https://seaborn.pydata.org/) facilita plotar alguns gráficos com mais detalhes.

In [ ]:
import seaborn as sns

sns.pairplot(df, hue=class_name)

# Scikit-learn

A biblioteca [scikit-learn](https://scikit-learn.org/) oferece diversas ferramentas para a mineração e análise de dados.

Ela possui ferramentas para pré-processamento, classificação, clusterização, regressão entre outras.

Tutoriais:
- https://www.datacamp.com/community/tutorials/machine-learning-python
- https://www.dataquest.io/blog/sci-kit-learn-tutorial/

Antes de qualquer coisa vamos primeiro separar os atributos preditivos do atributo classe

In [ ]:
X = df[feature_names]
y = df[class_name]

Podemos ver nas análises anteriores que nossa base não está normalizada.

Como normalizar esses dados?

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler_model = scaler.fit(X)

normalized_X = scaler_model.transform(X)

Será que os dados mudaram muito?

In [ ]:
X_df = pd.DataFrame(normalized_X, columns=feature_names)
X_df[class_name] = y
X_df.head()

In [ ]:
X_df.describe()

In [ ]:
sns.pairplot(X_df, hue=class_name)

Vamos treinar um modelo para classificaçãos utilizando árvore de decisão.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
model = dt.fit(normalized_X, y)

Algumas instâncias fictícias para testar o modelo.

In [ ]:
instances_to_predict = [[4.7,   2,   6.8,  0.3],
                        [4.5, 4.4, 2.05, 1.9],
                        [7,   2,   1.1,  2.2]]

In [ ]:
model.predict(instances_to_predict)

Notam algum problema?

Precisamos aplicar o mesmo pré-processamento para todas instâncias.

In [ ]:
normalized_instances = scaler_model.transform(instances_to_predict)
model.predict(normalized_instances)

In [ ]:
normalized_instances

# Avaliação

Nosso modelo funciona, mas ele é bom?

Precisamos avaliá-lo!

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt, normalized_X, y, cv=10, scoring="f1_micro")
print("Micro F1: ")
print("Valores: ", scores)
print("Média: ", sum(scores) / len(scores))

In [ ]:
scores = cross_val_score(dt, normalized_X, y, cv=10, scoring="f1_macro")
print("Macro F1: ")
print("Valores: ", scores)
print("Média: ", sum(scores) / len(scores))

# Avançado

É interessante juntar todas as tarefas utilizadas em uma pipeline para facilitar a utilização e parametrização.

In [ ]:
from sklearn.pipeline import Pipeline

steps = [("Scaler", scaler),
         ("Classifier", dt)]

full_pipeline = Pipeline(steps)

full_pipeline

Fazendo uma busca por melhores parametros.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'Classifier__criterion': ["gini", "entropy"],
              'Classifier__min_samples_leaf': [1, 2, 3],
              'Classifier__min_samples_split': [0.2, 0.5, 0.7]}

gscv = GridSearchCV(full_pipeline, param_grid, cv=10)

gscv.fit(X, y)

best_config = gscv.best_estimator_
print(best_config, gscv.best_score_)

Podemos utilizar essa métrica para julgar o modelo?

# Extra: eli5

A bliblioteca [eli5](https://eli5.readthedocs.io/) pode ajudar a descobrir por que um classificador classificou daquela forma.

In [ ]:
import eli5

eli5.show_weights(best_config, top=10, feature_names=feature_names,
                  target_names=best_config.classes_)